# Dask

`Dask` is an API that mimics `pandas`, however, a `dask` dataframe is distributed and operations against one is parallelized. The parallelization may be across a cluster of computers or your CPU cores.

In [1]:
import dask.dataframe as dd
import pandas as pd

In [2]:
pdf = pd.DataFrame({
    'x': [1, 2, 3],
    'y': [4, 5, 6]
})

ddf = dd.from_pandas(pdf, npartitions=2)

In [3]:
ddf

,x,y
npartitions=1,,
0,int64,int64
2,...,...


In [4]:
ddf.dtypes

x    int64
y    int64
dtype: object

In [5]:
ddf.describe()

,x,y
npartitions=1,,
,float64,float64
,...,...


In [6]:
ddf.sum().compute()

x     6
y    15
dtype: int64